In [58]:
import sympy as sp
from IPython.display import display, Math
import math as m
import time
start_time = time.time()

In [59]:
# x, y sú súradnice bodov v R^2
x, y = sp.symbols('x y')
# t je parameter z intervalu I
t = sp.Symbol('t')
# a, b sú konštanty pre škálovanie elipsoidu, a - dotykový smer, b - normálový smer
a, b = sp.symbols('a b')
# všetky ostatné symboly sú parametre v parametrizácii krivky parametric_curve
c, d = sp.symbols('c d')

In [60]:
# Vstupné parametre
# interval vyčíslenia krivky
t_min = -10
t_max = 10
# parametre škálovania
a_0 = 2
b_0 = 1
# parametre parametrizácie
c_0 = 2
d_0 = 1

# parametrizácia krivky
parametric_curve = sp.Matrix([sp.cos(t), sp.sin(t)]) 
#curve_name = 'parabola'

In [61]:
# Vypočíta dotyčnicu a normálu pre parametric_curve
def local_coordinate_system(parametric_curve): 
    t = sp.Symbol('t') 
    parametric_curve_first_derivation = sp.diff(parametric_curve, t)
    T = parametric_curve_first_derivation / sp.sqrt(parametric_curve_first_derivation.dot(parametric_curve_first_derivation))
    T = sp.simplify(T)
    N = sp.Matrix([-T[1], T[0]])
    N = sp.simplify(N)
    return T, N

# Vyrobí inverznú maticu prechodu 
def transition_matrix(T, N):
    P = sp.Matrix.hstack(T, N)
    return P.T

def matrix_of_quadrics_homogenization(A, B, C, D, E, F):
    row1 = [A, B/2, D/2]
    row2 = [B/2, C, E/2]
    row3 = [D/2, E/2, F]
    matrix = sp.Matrix([row1, row2, row3])
    return matrix

def matrix_of_quadrics(A, B, C):
    row1 = [A, B/2]
    row2 = [B/2, C]
    matrix = sp.Matrix([row1, row2])
    return matrix

In [62]:
# Výpočet vektorov
T, N = local_coordinate_system(parametric_curve)
# Zostavenie matice
P = transition_matrix(T, N)
print("Matica prechodu: ")
display(Math(sp.latex(P)))
# Maticou P vynásobíme vektor (X-parametric_curve) a tak zmeníme bázu elipsy
vector = sp.Matrix([x - parametric_curve[0], y - parametric_curve[1]])
result_vector = P * vector
print("Po transformácii: ")
display(Math(sp.latex(result_vector)))
ellipse_in_new_basis = result_vector[0]**2/a**2 + result_vector[1]**2/b**2 - 1
# Derivácia jednoparametrického systému elíps 
derivation = sp.diff(ellipse_in_new_basis, t)

Matica prechodu: 


<IPython.core.display.Math object>

Po transformácii: 


<IPython.core.display.Math object>

In [63]:
# Výpočet, o aký typ kužeľosečky ide
# Najprv potrebujeme extrahovať koeficienty z derivácie
expanded_expression = sp.expand(derivation)
A = expanded_expression.coeff(x**2)
B = expanded_expression.coeff(x*y)
C = expanded_expression.coeff(y**2)

# Tu bolo potrebné odstrániť všetky koeficienty pri členoch x*y, x*z,..
coef_x = expanded_expression.coeff(x)
terms_x = sp.Add(*coef_x.args) if isinstance(coef_x, sp.Add) else coef_x
terms_x_filtered = [term for term in sp.Add.make_args(terms_x) if not term.has(y)]
D = sp.Add(*terms_x_filtered)

coef_y = expanded_expression.coeff(y)
terms_y = sp.Add(*coef_y.args) if isinstance(coef_y, sp.Add) else coef_y
terms_y_filtered = [term for term in sp.Add.make_args(terms_y) if not term.has(x)]
E = sp.Add(*terms_y_filtered)

# Tu bolo potrebné odstrániť všetky koeficienty pri členoch x, y, z
terms = sp.Add(*expanded_expression.args) if isinstance(expanded_expression, sp.Add) else expanded_expression
terms_filtered = [term for term in sp.Add.make_args(terms) if not term.has(x) and not term.has(y)]
F = sp.Add(*terms_filtered)

matrix_of_derivation = matrix_of_quadrics_homogenization(A, B, C, D, E, F)
M1 = matrix_of_quadrics(A, B, C)
display(Math(sp.latex(M1)))

# Print koeficientov
print("Coefficient of x^2:")
display(Math(sp.latex(sp.simplify(A))))
print(sp.latex(sp.simplify(A)))
print("Coefficient of y^2:")
display(Math(sp.latex(sp.simplify(C))))
print(sp.latex(sp.simplify(C)))
print("Coefficient of xy:") 
display(Math(sp.latex(sp.simplify(B))))
print(sp.latex(sp.simplify(B)))
print("Coefficient of x:")
display(Math(sp.latex(sp.simplify(D))))
print(sp.latex(sp.simplify(D)))
print("Coefficient of y:")
display(Math(sp.latex(sp.simplify(E))))
print(sp.latex(sp.simplify(E)))
print("Coefficient of constant term:")
display(Math(sp.latex(sp.simplify(F))))
print(sp.latex(sp.simplify(F)))

<IPython.core.display.Math object>

Coefficient of x^2:


<IPython.core.display.Math object>

- \frac{\sin{\left(2 t \right)}}{b^{2}} + \frac{\sin{\left(2 t \right)}}{a^{2}}
Coefficient of y^2:


<IPython.core.display.Math object>

...

\frac{2 \sin{\left(t \right)}}{b^{2}}
Coefficient of y:


<IPython.core.display.Math object>

- \frac{2 \cos{\left(t \right)}}{b^{2}}
Coefficient of constant term:


<IPython.core.display.Math object>

0


In [64]:
# Typ kužeľosečky
determinant = sp.det(matrix_of_derivation)
subdeterminant = A*C-B**2/4

In [65]:
# Diagonalize the matrix
P1, D1 = M1.diagonalize()

subdet = D1.det()
print("Determinant matice kvadratickej formy: ")
display(Math(sp.latex(sp.simplify(subdet))))

print("Diagonálna matica: ")
display(Math(sp.latex(sp.simplify(D1))))

print("Vlastné hodnoty: ")
print(sp.latex(sp.simplify(D1[0,0])))
print(sp.latex(sp.simplify(D1[1,1])))

print("Matica vlastných vektorov: ")
display(Math(sp.latex(P1)))

Determinant matice kvadratickej formy: 


<IPython.core.display.Math object>

Diagonálna matica: 


<IPython.core.display.Math object>

Vlastné hodnoty: 
- \frac{1}{b^{2}} + \frac{1}{a^{2}}
\frac{1}{b^{2}} - \frac{1}{a^{2}}
Matica vlastných vektorov: 


<IPython.core.display.Math object>

In [69]:
# kvadratická forma tvaru Ax^2+Bxy + Cy^2 + Dx + Ey + F = 0
phi = 1/2 * sp.atan(B/(C-A))
display(Math(sp.latex(sp.simplify(phi))))
x_1 = sp.cos(phi)*x - sp.sin(phi)*y
y_1 = sp.sin(phi)*x + sp.cos(phi)*y
display(Math(sp.latex(sp.simplify(x_1))))
display(Math(sp.latex(sp.simplify(y_1))))
x_11 = x_1 + (D*sp.cos(phi) - E*sp.sin(phi))/D1[1,1]
y_11 = y_1 + (D*sp.sin(phi) + E*sp.cos(phi))/D1[0,0]
display(Math(sp.latex(sp.simplify(x_11))))
display(Math(sp.latex(sp.simplify(y_11))))
eq = D1[1,1]*x_11**2 + D1[0,0]*y_11**2
eq_simp = sp.latex(sp.simplify(eq))
display(Math(eq_simp))
print(eq_simp)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

\frac{\left(2 a^{2} \sin{\left(t + 0.5 \operatorname{atan}{\left(\frac{1}{\tan{\left(2 t \right)}} \right)} \right)} + \left(a - b\right) \left(a + b\right) \left(x \cos{\left(0.5 \operatorname{atan}{\left(\frac{1}{\tan{\left(2 t \right)}} \right)} \right)} - y \sin{\left(0.5 \operatorname{atan}{\left(\frac{1}{\tan{\left(2 t \right)}} \right)} \right)}\right)\right)^{2} - \left(2 a^{2} \cos{\left(t + 0.5 \operatorname{atan}{\left(\frac{1}{\tan{\left(2 t \right)}} \right)} \right)} + \left(a - b\right) \left(a + b\right) \left(x \sin{\left(0.5 \operatorname{atan}{\left(\frac{1}{\tan{\left(2 t \right)}} \right)} \right)} + y \cos{\left(0.5 \operatorname{atan}{\left(\frac{1}{\tan{\left(2 t \right)}} \right)} \right)}\right)\right)^{2}}{a^{2} b^{2} \left(a - b\right) \left(a + b\right)}


In [67]:
# smerové vektory hlavných osí
# Define eigenvectors (replace with your expressions)
eigen_vector0 = sp.sqrt(sp.Abs(D[0, 0])) * P[:, 0]
eigen_vector1 = sp.sqrt(sp.Abs(D[1, 1])) * P[:, 1]
eigen_vector2 = sp.sqrt(sp.Abs(D[2, 2])) * P[:, 2]
#display(Math(sp.latex(eigen_vector1)))
#display(Math(sp.latex(eigen_vector2)))

# Define a tolerance value for considering a number as zero
tolerance = 1e-15  # You can adjust this tolerance as needed

# Check if each eigenvector is effectively zero
is_nonzero0 = abs(D[0,0]) >= tolerance 
is_nonzero1 = abs(D[1,1]) >= tolerance 
is_nonzero2 = abs(D[2,2]) >= tolerance

# Determine which eigenvectors are nonzero
nonzero_eigenvalues = []
nonzero_eigenvectors = []
if is_nonzero0:
    nonzero_eigenvalues.append(D[0,0])
    #nonzero_eigenvectors.append(eigen_vector0/sp.sqrt(eigen_vector0[0]**2 + eigen_vector0[1]**2 + eigen_vector0[2]**2))
    nonzero_eigenvectors.append(eigen_vector0)
if is_nonzero1:
    nonzero_eigenvalues.append(D[1,1])
    nonzero_eigenvectors.append(eigen_vector1)
    #nonzero_eigenvectors.append(eigen_vector1/sp.sqrt(eigen_vector1[0]**2 + eigen_vector1[1]**2 + eigen_vector1[2]**2))
if is_nonzero2:
    nonzero_eigenvalues.append(D[2,2])
    nonzero_eigenvectors.append(eigen_vector2)
    #nonzero_eigenvectors.append(eigen_vector2/sp.sqrt(eigen_vector2[0]**2 + eigen_vector2[1]**2 + eigen_vector2[2]**2))

#print(len(nonzero_eigenvectors))

# po sčítaní osí dostávam žiadaný smer
Q1 = (nonzero_eigenvectors[0] + nonzero_eigenvectors[1])
if (nonzero_eigenvalues[1] >= nonzero_eigenvalues[0]):
    Q2 =(nonzero_eigenvectors[1] - nonzero_eigenvectors[0])
else:
    Q2 = (nonzero_eigenvectors[0] - nonzero_eigenvectors[1])

g = Q1[0]*x+Q1[1]*y+Q1[2]
h = Q2[0]*x+Q2[1]*y+Q2[2]
print(sp.latex(g) + ' = 0')
print(sp.latex(h) + ' = 0')

theta1 = m.atan2(Q1[1],Q1[0])
d1 = Q1[2]/ m.sqrt(Q1[0]**2+Q1[1]**2)
theta2 = m.atan2(Q2[1],Q2[0])
d2 = Q2[2]/ m.sqrt(Q2[0]**2+Q2[1]**2)

lineg = sp.cos(theta1)*x + sp.sin(theta1)*y + d1
lineh = sp.cos(theta2)*x + sp.sin(theta2)*y + d2

print(sp.latex(lineg))
print(sp.latex(lineh))

TypeError: 'Add' object is not subscriptable

In [ ]:
# Ukončenie merania času
end_time = time.time()
elapsed_time = end_time - start_time

In [ ]:
# Typ kuželosečky vieme jasne dourčiť pre špecifikáciu parametrov
t_values = range(t_min, t_max)
is_negative = True

for t_val in t_values:
    if subdeterminant.subs({t: t_val, a: a_0, b: b_0, c: c_0, d: d_0}) >= 0:
        is_negative = False
        break

if is_negative:
    print("Výraz je pre zadané hodnoty záporný. ")
else:
    print("Výraz nie je pre zadané hodnoty záporný. ")

Výraz je pre zadané hodnoty záporný. 


In [ ]:
print(parametric_curve)
print("Systém: ")
system = sp.latex(sp.simplify(ellipse_in_new_basis))
display(Math(system))
#print(system + '=0')

print("Derivácia sytému: ")
der = sp.latex(sp.simplify(derivation))
display(Math(der))
#print(der + '=0')

print("Determinant derivácie: ")
display(Math(sp.latex(sp.simplify(determinant))))

print("Čas: ")
print(elapsed_time, "seconds")

expression = sp.latex(sp.simplify(subdeterminant))
display(Math(expression))
if is_negative:
    print("Výraz je pre zadané hodnoty záporný. ")
else:
    print("Výraz nie je pre zadané hodnoty záporný. ")

Matrix([[t], [t**2]])
Systém: 


<IPython.core.display.Math object>

Derivácia sytému: 


<IPython.core.display.Math object>

Determinant derivácie: 


<IPython.core.display.Math object>

Čas: 
2.349496364593506 seconds


<IPython.core.display.Math object>

Výraz je pre zadané hodnoty záporný. 
